In [13]:
import cx_Oracle
import os
import json
import pandas as pd

In [14]:
user_bd = "admin"
password_bd = "admin"
tns_bd = "DKR_NAC_213"

In [15]:
def database_conection(user_bd='admin', password_bd='admin', tns_bd='DKR_NAC_213'):
    try:
        conn = cx_Oracle.connect(user=str(user_bd), password=str(password_bd), dsn=str(tns_bd))
        cursor = conn.cursor()
        return conn, cursor
    except cx_Oracle.DatabaseError as e:
        return None, None


In [16]:
def get_data(query, cursor):
    '''
        Esta função faz requisições no bano de dados, criando dataframe com as informações obtidas
    '''
    try:
        cursor.execute(query)
        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        
        df = pd.DataFrame(results, columns=columns)
        return df

    except cx_Oracle.DatabaseError as e:
        return None

In [17]:
def make_db_highq_login(cursor):

    #script_dir = os.path.dirname(__file__)
    file_path = os.path.join('config/query.json')

    with open(file_path) as file:
        querys = json.load(file)

    
    df_login_user_data = get_data(querys['loginUserData'], cursor)
    df_user_bd = get_data(querys['userBD'], cursor)
    return df_login_user_data, df_user_bd

In [18]:
def make_db_highq_users_bd(cursor):

    #script_dir = os.path.dirname(__file__)
    file_path = os.path.join('config/query.json')

    with open(file_path) as file:
        querys = json.load(file)

    
    df_user_bd = get_data(querys['userBD'], cursor)
    return df_user_bd

In [19]:
def make_db_register(cursor):

    #script_dir = os.path.dirname(__file__)
    file_path = os.path.join('config/query.json')

    with open(file_path) as file:
        querys = json.load(file)
    
    df_estados = get_data(querys['estadosBR'], cursor)
    df_class_process = get_data(querys['classProcess'], cursor)
    df_path_process = get_data(querys['pathProcess'], cursor)
    return df_estados, df_class_process, df_path_process


In [20]:
conn, cursor = database_conection(user_bd, password_bd, tns_bd)
df_login_user_data, df_user_bd = make_db_highq_login(cursor)
df_estados, df_class_process, df_path_process = make_db_register(cursor)


In [21]:
df_path_process.head()

,CAMINHO_PROCESSUAL
0,Apelação
1,Conciliação
2,Contestação
3,Intimação
4,Notificação


In [23]:
sigla_estados_list = df_estados['SIGLA_ESTADO'].tolist()
class_process_list = df_class_process['CLASSE_PROCESSO'].tolist()
path_process_list = df_path_process['CAMINHO_PROCESSUAL'].tolist()

In [26]:
print(class_process_list)

['Ação Administrativa', 'Ação Civil', 'Ação Constitucional', 'Ação Empresarial', 'Ação Penal', 'Ação Trabalhista', 'Ação Tributária']


In [24]:
df_login_user_data.head()

,ID,NAME,EMAIL,LOGIN_PASSWORD,NUM_OAB,USERNAME
0,1,admin,admin@gmail.com,1234,0099887766,admin
1,2,Teste de cadastro,teste@gmail.com,123,00000000,teste
2,3,Teste de cadastro 1.2,teste1@gmail.com,123,00000001,teste1
3,8,teste 02,teste2@gmail.com,teste,000000002,teste2


In [25]:
df_user_bd.head()

,ID,USERNAME,PASSWORD,DATA_CREIACAO
0,1,admin,admin,2025-06-18 01:19:27
1,2,teste1,teste1,2025-06-20 01:38:00
2,3,teste2,teste2,2025-06-20 01:40:05


In [ ]:
#Receber o email e senha do usuário através da tela 
username = 'admin@gmail.com'
password = '1234'

#Entrar com o usuário do banco admin para verificar se o usuário solicitando login existe no banco de dados para pegar as credenciais de banco para se conectar
conn, cursor = database_conection()
df_login_user_data, df_user_bd = make_db_highq_login(cursor)
if username in df_login_user_data['EMAIL'].to_list() and password in df_login_user_data['LOGIN_PASSWORD'].to_list():
    print("Usuário encontrado no banco de dados.")
    user_bd_login = df_login_user_data[df_login_user_data['EMAIL'] == username]['USERNAME'].values[0]
    password_bd_login = df_user_bd[df_user_bd['USERNAME'] == user_bd_login]['PASSWORD'].values[0]
    print(f"Username: {user_bd_login}, Password: {password_bd_login}")
else: print("Usuário ou senha inválidos.")
    

Usuário encontrado no banco de dados.
Username: admin, Password: admin
